## CNN
### CIFAR100_VGG13

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Sequential, datasets, optimizers
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
# 使用allow_growth option，刚一开始分配少量的GPU容量，然后按需慢慢的增加，动态申请显存
# 由于不会释放内存，所以会导致碎片
config = ConfigProto()   # 用在创建session的时候，用来对session进行参数配置
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config)

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.random.set_seed(2345)

In [4]:
# convolution layers
conv_layers = [
    # 5 units of conv + max poling
    # unit 1 
    # Conv2D()中第一个参数表示经过这一层的卷积操作后输出的通道数
    layers.Conv2D(64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    # unit 2
    layers.Conv2D(128, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    # unit 3
    layers.Conv2D(256, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    # unit 4
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    # unit 5
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same')
    
]
conv_net = Sequential(conv_layers)

# full connection layers
fc_net = Sequential([
    layers.Dense(512, activation=tf.nn.relu),    # [b, 512] => [b, 512]
    layers.Dense(256, activation=tf.nn.relu),    # [b, 512] => [b, 256]
    layers.Dense(100, activation=None)           # [b, 256] => [b, 100]
])


In [5]:
# 定义预处理函数
def preprocess(x, y):
    # [0~1]
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y
# 创建网络层
conv_net.build(input_shape=[None, 32, 32, 3])
fc_net.build(input_shape=[None, 512])
# 定义优化器
optimizer = optimizers.Adam(lr=1e-4)
# 定义网络成参数列表
variables = conv_net.trainable_variables + fc_net.trainable_variables


In [6]:
# 加载数据集和数据集预处理
(x, y), (x_test, y_test) = datasets.cifar100.load_data()
y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.shuffle(10000).map(preprocess).batch(256)
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(256)

# 查看数据集切片
sample = next(iter(train_db))
print('sample_train: ', sample[0].shape, sample[1].shape, tf.reduce_min(sample[0]),
     tf.reduce_max(sample[1]))

sample_train:  (256, 32, 32, 3) (256,) tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(99, shape=(), dtype=int32)


In [7]:
for epoch in range(20):
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            out = conv_net(x)    # [b, 32, 32, 32, 3] => [b, 1, 1, 512]
            out = tf.reshape(out, [-1, 512])    # [b, 1, 1, 512] => [b, 512]
            logits = fc_net(out)    # [b, 512] => [b, 100]
            y_onehot = tf.one_hot(y, depth=100)
            # compute loss
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        
        # compute grades    
        grads = tape.gradient(loss, variables)
        # update variables 
        optimizer.apply_gradients(zip(grads, variables))
        
        # show loss
        if step % 100 == 0:
            print(epoch, step, 'loss: ', float(loss))
            
    # test    
    total_num = 0
    total_correct = 0
    for x, y in test_db:
        out = conv_net(x)
        out = tf.reshape(out, [-1, 512])
        logits = fc_net(out)
        prob = tf.nn.softmax(logits, axis=1)
        preb = tf.argmax(prob, axis=1)    # int64
        preb = tf.cast(preb, dtype=tf.int32)
        
        correct = tf.cast(tf.equal(preb, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)
        
        total_num += x.shape[0]
        total_correct += int(correct)
        
    acc = total_correct / total_num
    print(epoch, 'acc: ', acc)

0 0 loss:  4.605621814727783
0 100 loss:  4.398139953613281
0 acc:  0.054
1 0 loss:  4.107680320739746
1 100 loss:  4.153841018676758
1 acc:  0.0811
2 0 loss:  3.950089931488037
2 100 loss:  3.9207491874694824
2 acc:  0.1235
3 0 loss:  3.746234178543091
3 100 loss:  3.6879868507385254
3 acc:  0.1596
4 0 loss:  3.539954423904419
4 100 loss:  3.4683966636657715
4 acc:  0.1824
5 0 loss:  3.3869028091430664
5 100 loss:  3.3026139736175537
5 acc:  0.2104
6 0 loss:  3.1666555404663086
6 100 loss:  3.1883835792541504
6 acc:  0.2304
7 0 loss:  2.9812355041503906
7 100 loss:  3.0427980422973633
7 acc:  0.2393
8 0 loss:  2.8564555644989014
8 100 loss:  2.9154019355773926
8 acc:  0.241
9 0 loss:  2.7748284339904785
9 100 loss:  2.783421754837036
9 acc:  0.2413
10 0 loss:  2.6757168769836426
10 100 loss:  2.6323559284210205
10 acc:  0.2473
11 0 loss:  2.572598695755005
11 100 loss:  2.4826223850250244
11 acc:  0.2516
12 0 loss:  2.5238289833068848
12 100 loss:  2.41087007522583
12 acc:  0.2573
13 